# Daily Observations: The Productivity Mirage — Why This Cycle Is Different

---

*A systematic macro research memo*

**Date:** February 2026  
**Author:** Hafsa Ghannaj  
**Classification:** Research — For Discussion

---

## Executive Summary

The prevailing market narrative holds that artificial intelligence will deliver a productivity boom analogous to the 1990s — implying that today's elevated equity multiples are justified by an incoming wave of disinflationary growth. **We believe this view is wrong, and dangerously so.**

The 1990s productivity miracle was not primarily a technology story. It was a *globalization* story. The integration of 800 million Chinese workers into the global labor force, combined with falling energy costs and post-Cold War fiscal restraint, created a once-in-history deflationary tailwind that amplified the impact of the internet revolution. Technology was the spark, but cheap globalization was the oxygen.

Today, that oxygen is gone. The AI revolution is unfolding in a world of:
- **Deglobalization and onshoring** (CHIPS Act, friend-shoring, tariff escalation)
- **Structurally higher energy costs** (energy transition, underinvestment in hydrocarbons)
- **Fiscal dominance** (debt/GDP >120%, persistent deficits, politicized monetary policy)
- **Labor scarcity** (demographics, immigration restrictions, unionization)

We systematize this thesis into a quantitative `ProductivityRegimeIndicator` that scores the macro environment on a scale from -1 (deflationary productivity regime, like the 1990s) to +1 (inflationary productivity regime, like today). We then show that equity multiples are historically sensitive to this indicator — and that current pricing implies a regime that does not exist.

---

## Section 1: What Happened Then

Between 1990 and 2000, the US experienced what many economists call the "Great Moderation" on steroids:

- **Real GDP growth** averaged 3.8% per year
- **Core PCE inflation** fell from ~5% to ~1.5%
- **The federal budget** went from deficit to surplus
- **The S&P 500** returned ~430% (total return)
- **The Shiller CAPE** expanded from 17x to 44x

The standard narrative attributes this to the internet and information technology. But technology adoption alone cannot explain the simultaneous decline in inflation. Productivity booms should increase *demand* as well as supply — the net effect on prices is ambiguous.

What made the 1990s unique was that technology-driven productivity gains were *reinforced* by three exogenous deflationary forces:

1. **China's WTO accession process (1990s):** China's export prices fell relentlessly as hundreds of millions of workers moved from subsistence farming to manufacturing. This suppressed global goods prices directly.

2. **Energy abundance:** Oil prices averaged ~$20/barrel through most of the 1990s. The Soviet collapse flooded markets with cheap energy.

3. **Fiscal consolidation:** The Clinton administration, working with a Republican Congress, eliminated the budget deficit. This kept real yields contained even as growth accelerated.

The combination was extraordinary: rising productivity + falling input costs + fiscal discipline = non-inflationary growth. The Fed could keep rates relatively low, and equity multiples expanded to reflect a perceived permanent shift in the economy's potential.

**The key insight: the 1990s multiple expansion was not a *technology* premium — it was a *globalization* premium.**

---

## Section 2: What's Different Now

Every structural tailwind of the 1990s has reversed:

| Factor | 1990s | 2020s |
|--------|-------|-------|
| China | Deflationary exporter, integrating into global supply chains | Strategic competitor, subject to tariffs and export controls |
| Energy | $20/bbl oil, Soviet supply glut | Energy transition costs, OPEC+ discipline, underinvestment |
| Fiscal | Budget surpluses by 1998 | Debt/GDP >120%, $2T+ deficits, no political will for consolidation |
| Labor | Growing workforce, low unionization, immigration | Aging demographics, rising unionization, immigration restrictions |
| Monetary | Independent Fed, credible inflation target | Fiscal dominance risk, politicization pressure |
| Geopolitics | Unipolar US hegemony, "peace dividend" | Multipolar fragmentation, rising defense spending |

This means that even if AI delivers *identical* productivity gains to the internet, the macro transmission mechanism is fundamentally different. Productivity gains in an inflationary environment get absorbed by rising input costs rather than flowing through to margin expansion and multiple re-rating.

**Historical analogy:** The 1960s also saw rapid technological progress (space race, early computing, manufacturing automation). But because it occurred alongside fiscal expansion (Vietnam, Great Society) and tight labor markets, the productivity gains were inflated away. The S&P 500's real return from 1966 to 1982 was approximately *zero*.

We are not predicting stagflation. We are arguing that the *composition* of the current macro regime means productivity gains will not translate into the disinflation and multiple expansion that investors are pricing.

---

## Section 3: How We Systematize This

To move from narrative to algorithm, we construct a **Productivity Regime Indicator** that captures whether the macro environment is conducive to deflationary (1990s-style) or inflationary (current) productivity transmission.

The indicator synthesizes three sub-signals:

1. **Globalization Pressure** — proxied by China export/producer prices. Falling = deflationary globalization; rising = inflationary deglobalization.

2. **Real Yield Stability** — calculated as the volatility of real yields (nominal 10Y minus trailing PCE inflation). Low and stable = benign financial conditions; high and volatile = tightening or fiscal stress.

3. **Labor Cost Pressure** — proxied by US unit labor costs. Falling = labor market slack (1990s immigration + globalization); rising = structural labor scarcity.

Each component is z-scored against its own history and combined into a composite score bounded between -1 and +1:

$$\text{Regime Score} = \tanh\left(\frac{z_{\text{china}} + z_{\text{real\_yield\_vol}} + z_{\text{ulc}}}{3}\right)$$

- Score near **-1**: Deflationary productivity regime (1990s). Globalization suppressing prices, yields stable, labor cheap.
- Score near **0**: Neutral.
- Score near **+1**: Inflationary productivity regime (current). Deglobalization, volatile yields, expensive labor.

We then examine whether this indicator has predictive power for equity multiple changes.

---

In [ ]:
# ============================================================
# SETUP AND IMPORTS
# ============================================================

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.gridspec import GridSpec
from datetime import datetime
from scipy.stats import pearsonr

import yfinance as yf

# FRED API setup — set your key as environment variable: export FRED_API_KEY="your_key"
# If no key is available, we fall back to synthetic proxies built from public data.
FRED_API_KEY = os.environ.get('FRED_API_KEY', None)
USE_FRED = False

if FRED_API_KEY:
    try:
        from fredapi import Fred
        fred = Fred(api_key=FRED_API_KEY)
        USE_FRED = True
        print("FRED API connected successfully.")
    except Exception as e:
        print(f"FRED API connection failed: {e}. Using fallback data construction.")
else:
    print("No FRED_API_KEY found. Using fallback data construction from public sources.")

# Plotting configuration
plt.rcParams.update({
    'figure.facecolor': '#0a0a0a',
    'axes.facecolor': '#0a0a0a',
    'axes.edgecolor': '#333333',
    'axes.labelcolor': '#cccccc',
    'text.color': '#cccccc',
    'xtick.color': '#999999',
    'ytick.color': '#999999',
    'grid.color': '#1a1a1a',
    'grid.alpha': 0.8,
    'font.family': 'monospace',
    'font.size': 10,
    'figure.dpi': 120,
    'savefig.dpi': 150,
    'figure.figsize': (14, 7),
})

# Color palette
C_ORANGE = '#FF6B00'
C_BLUE = '#4A90D9'
C_GREEN = '#2ECC71'
C_RED = '#E74C3C'
C_PURPLE = '#9B59B6'
C_YELLOW = '#F1C40F'
C_GRAY = '#7F8C8D'
C_WHITE = '#ECF0F1'

print("Setup complete.")

In [ ]:
# ============================================================
# DATA ACQUISITION
# ============================================================

START_DATE = '1988-01-01'
END_DATE = datetime.today().strftime('%Y-%m-%d')

print("Fetching market data from Yahoo Finance...")

# S&P 500
sp500 = yf.download('^GSPC', start=START_DATE, end=END_DATE, progress=False)
# Handle both single and multi-level columns from yfinance
if isinstance(sp500.columns, pd.MultiIndex):
    sp500.columns = sp500.columns.get_level_values(0)
sp500_monthly = sp500['Close'].resample('MS').last().rename('SP500')

# 10-Year Treasury Yield
tnx = yf.download('^TNX', start=START_DATE, end=END_DATE, progress=False)
if isinstance(tnx.columns, pd.MultiIndex):
    tnx.columns = tnx.columns.get_level_values(0)
tnx_monthly = tnx['Close'].resample('MS').last().rename('TNX') / 100  # Convert to decimal

# Semiconductor ETF (AI proxy) — SMH available from ~2000
smh = yf.download('SMH', start='1999-01-01', end=END_DATE, progress=False)
if isinstance(smh.columns, pd.MultiIndex):
    smh.columns = smh.columns.get_level_values(0)
smh_monthly = smh['Close'].resample('MS').last().rename('SMH')

print(f"  S&P 500: {sp500_monthly.index[0].strftime('%Y-%m')} to {sp500_monthly.index[-1].strftime('%Y-%m')} ({len(sp500_monthly)} months)")
print(f"  10Y Yield: {tnx_monthly.index[0].strftime('%Y-%m')} to {tnx_monthly.index[-1].strftime('%Y-%m')} ({len(tnx_monthly)} months)")
print(f"  SMH: {smh_monthly.index[0].strftime('%Y-%m')} to {smh_monthly.index[-1].strftime('%Y-%m')} ({len(smh_monthly)} months)")

In [ ]:
# ============================================================
# FRED DATA (with fallback construction)
# ============================================================

if USE_FRED:
    print("Fetching macro data from FRED...")
    
    # PCE Price Index
    pcepi = fred.get_series('PCEPI', observation_start=START_DATE).resample('MS').last().rename('PCEPI')
    
    # Unit Labor Costs — Nonfarm Business (index, quarterly, interpolate to monthly)
    ulc = fred.get_series('ULCNBS', observation_start=START_DATE).rename('ULC')
    ulc = ulc.resample('MS').interpolate(method='linear')
    
    # China CPI (proxy for export price pressure) — FRED series
    china_cpi = fred.get_series('CHNCPIALLMINMEI', observation_start=START_DATE).resample('MS').last().rename('ChinaCPI')
    
    print("  FRED data loaded successfully.")
    
else:
    print("Constructing macro proxies from public data...")
    print("  (For production use, set FRED_API_KEY for authoritative data)")
    
    # Construct proxy series using well-documented historical values.
    # These are calibrated to match FRED series characteristics.
    
    date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='MS')
    
    # --- PCE Price Index proxy ---
    # PCE index: base ~60 in 1988, ~75 in 2000, ~82 in 2005, ~100 in 2012, ~117 in 2020, ~130 in 2024
    # We model this as a smooth path through known anchor points.
    pce_anchors = {
        '1988-01-01': 60.0, '1990-01-01': 64.0, '1992-01-01': 68.0,
        '1995-01-01': 72.0, '1998-01-01': 75.0, '2000-01-01': 77.5,
        '2003-01-01': 81.0, '2006-01-01': 87.0, '2008-01-01': 92.0,
        '2009-01-01': 91.0, '2010-01-01': 93.0, '2012-01-01': 96.5,
        '2014-01-01': 99.5, '2015-01-01': 100.0, '2016-01-01': 101.0,
        '2018-01-01': 104.5, '2020-01-01': 107.0, '2021-01-01': 110.0,
        '2022-01-01': 118.5, '2023-01-01': 125.0, '2024-01-01': 128.5,
        '2025-01-01': 131.0, '2026-01-01': 133.5,
    }
    pce_idx = pd.Series(pce_anchors).rename('PCEPI')
    pce_idx.index = pd.to_datetime(pce_idx.index)
    pcepi = pce_idx.reindex(date_range).interpolate(method='cubic').rename('PCEPI')
    
    # --- Unit Labor Costs proxy ---
    # ULC index (2012=100): ~70 in 1988, relatively flat 1995-2005 (~80-90),
    # then accelerating post-2020.
    ulc_anchors = {
        '1988-01-01': 68.0, '1990-01-01': 72.0, '1993-01-01': 76.0,
        '1996-01-01': 79.0, '2000-01-01': 83.0, '2003-01-01': 87.0,
        '2006-01-01': 90.0, '2008-01-01': 94.0, '2009-01-01': 98.0,
        '2010-01-01': 96.0, '2012-01-01': 100.0, '2015-01-01': 104.0,
        '2018-01-01': 109.0, '2020-01-01': 116.0, '2021-01-01': 121.0,
        '2022-01-01': 130.0, '2023-01-01': 137.0, '2024-01-01': 142.0,
        '2025-01-01': 146.0, '2026-01-01': 149.0,
    }
    ulc_s = pd.Series(ulc_anchors).rename('ULC')
    ulc_s.index = pd.to_datetime(ulc_s.index)
    ulc = ulc_s.reindex(date_range).interpolate(method='cubic').rename('ULC')
    
    # --- China CPI / Export Price Pressure proxy ---
    # China CPI index (2010=100): rapid growth in 1990s from low base, disinflation
    # in early 2000s, reflation post-2009, recent stagnation/deflation.
    china_anchors = {
        '1988-01-01': 42.0, '1990-01-01': 48.0, '1993-01-01': 58.0,
        '1995-01-01': 68.0, '1997-01-01': 73.0, '1999-01-01': 71.0,
        '2001-01-01': 72.0, '2003-01-01': 73.0, '2005-01-01': 76.0,
        '2007-01-01': 80.0, '2008-01-01': 85.0, '2009-01-01': 83.0,
        '2010-01-01': 86.0, '2011-01-01': 91.0, '2013-01-01': 96.0,
        '2015-01-01': 99.0, '2016-01-01': 101.0, '2018-01-01': 104.0,
        '2020-01-01': 108.0, '2021-01-01': 109.0, '2022-01-01': 112.0,
        '2023-01-01': 112.5, '2024-01-01': 112.0, '2025-01-01': 112.5,
        '2026-01-01': 113.0,
    }
    china_s = pd.Series(china_anchors).rename('ChinaCPI')
    china_s.index = pd.to_datetime(china_s.index)
    china_cpi = china_s.reindex(date_range).interpolate(method='cubic').rename('ChinaCPI')
    
    print("  Proxy data constructed successfully.")

# Combine into a single DataFrame
macro = pd.DataFrame({
    'SP500': sp500_monthly,
    'TNX': tnx_monthly,
    'PCEPI': pcepi,
    'ULC': ulc,
    'ChinaCPI': china_cpi,
}).dropna(how='all')

# Forward-fill short gaps, then drop remaining NaNs at edges
macro = macro.ffill(limit=3).dropna()

print(f"\nCombined dataset: {macro.index[0].strftime('%Y-%m')} to {macro.index[-1].strftime('%Y-%m')} ({len(macro)} months)")
macro.tail()

In [ ]:
# ============================================================
# DERIVED FEATURES
# ============================================================

# PCE YoY inflation rate
macro['PCE_YoY'] = macro['PCEPI'].pct_change(12)

# Real yield = nominal 10Y - trailing 12m PCE inflation
macro['RealYield'] = macro['TNX'] - macro['PCE_YoY']

# Rolling 12-month volatility of real yields (annualized std of monthly changes)
macro['RealYield_Vol'] = macro['RealYield'].diff().rolling(12).std() * np.sqrt(12)

# China CPI YoY change (proxy for export price pressure)
macro['China_YoY'] = macro['ChinaCPI'].pct_change(12)

# Unit labor cost YoY change
macro['ULC_YoY'] = macro['ULC'].pct_change(12)

# S&P 500 trailing 12-month earnings yield proxy (1/PE using 10Y trailing returns)
macro['SP500_12m_Return'] = macro['SP500'].pct_change(12)

macro = macro.dropna()
print(f"Feature-enriched dataset: {macro.index[0].strftime('%Y-%m')} to {macro.index[-1].strftime('%Y-%m')} ({len(macro)} obs)")

In [ ]:
# ============================================================
# SHILLER CAPE RATIO (from Robert Shiller's public dataset)
# ============================================================

print("Fetching Shiller CAPE data...")

try:
    # Shiller's data is publicly available as an Excel file
    shiller_url = 'http://www.econ.yale.edu/~shiller/data/ie_data.xls'
    shiller_raw = pd.read_excel(shiller_url, sheet_name='Data', header=7)
    
    # The date column is formatted as YYYY.MM (e.g., 1990.01)
    shiller_raw = shiller_raw.dropna(subset=[shiller_raw.columns[0]])
    
    # Extract date and CAPE columns
    date_col = shiller_raw.columns[0]
    cape_col = 'CAPE'  # Shiller names it CAPE or P10 or similar
    
    # Find the CAPE column (it may have different names)
    cape_candidates = [c for c in shiller_raw.columns if 'cape' in str(c).lower() or 'p10' in str(c).lower()]
    if cape_candidates:
        cape_col = cape_candidates[0]
    else:
        # CAPE is typically the last or second-to-last column
        cape_col = shiller_raw.columns[-2]
    
    # Parse dates from the float format YYYY.MM
    dates = []
    capes = []
    for _, row in shiller_raw.iterrows():
        try:
            d = float(row[date_col])
            c = float(row[cape_col])
            year = int(d)
            month = round((d - year) * 100)
            if month == 0:
                month = 1
            if 1880 <= year <= 2030 and 1 <= month <= 12 and 0 < c < 200:
                dates.append(pd.Timestamp(year=year, month=month, day=1))
                capes.append(c)
        except (ValueError, TypeError):
            continue
    
    cape_series = pd.Series(capes, index=dates, name='CAPE').sort_index()
    cape_series = cape_series[~cape_series.index.duplicated(keep='last')]
    print(f"  Shiller CAPE loaded: {cape_series.index[0].strftime('%Y-%m')} to {cape_series.index[-1].strftime('%Y-%m')}")
    
except Exception as e:
    print(f"  Shiller download failed ({e}). Constructing CAPE proxy...")
    
    # Well-documented CAPE values at key dates
    cape_anchors = {
        '1988-01-01': 14.5, '1990-01-01': 17.0, '1993-01-01': 20.0,
        '1995-01-01': 21.0, '1997-01-01': 28.0, '1999-01-01': 40.0,
        '2000-03-01': 44.0, '2002-10-01': 22.0, '2004-01-01': 26.0,
        '2007-10-01': 27.5, '2009-03-01': 13.3, '2011-01-01': 22.0,
        '2013-01-01': 22.5, '2015-01-01': 27.0, '2018-01-01': 33.0,
        '2018-12-01': 27.0, '2020-02-01': 31.0, '2020-03-01': 24.0,
        '2021-12-01': 39.0, '2022-10-01': 28.0, '2023-06-01': 31.0,
        '2024-01-01': 34.0, '2024-06-01': 36.0, '2025-01-01': 37.5,
        '2025-12-01': 36.0, '2026-01-01': 35.5,
    }
    cape_s = pd.Series(cape_anchors, name='CAPE', dtype=float)
    cape_s.index = pd.to_datetime(cape_s.index)
    date_range = pd.date_range(start='1988-01-01', end=END_DATE, freq='MS')
    cape_series = cape_s.reindex(date_range).interpolate(method='cubic').rename('CAPE')

# Align CAPE with our macro dataframe
macro['CAPE'] = cape_series.reindex(macro.index).ffill()
macro = macro.dropna(subset=['CAPE'])

print(f"  CAPE aligned: {macro.index[0].strftime('%Y-%m')} to {macro.index[-1].strftime('%Y-%m')}")

In [ ]:
# ============================================================
# THE CORE ALGORITHM: ProductivityRegimeIndicator
# ============================================================

class ProductivityRegimeIndicator:
    """
    Classifies the macro environment into deflationary vs. inflationary
    productivity regimes using three observable signals:
    
    1. Globalization pressure (China export/CPI prices)
    2. Real yield volatility (financial conditions stress)
    3. Unit labor cost growth (labor market tightness)
    
    Output: continuous score from -1 (deflationary, 1990s-like)
            to +1 (inflationary, current regime)
    
    All calculations use trailing data only — no look-ahead bias.
    """
    
    def __init__(self, lookback_months: int = 60):
        self.lookback = lookback_months
        self.components_ = None
        self.score_ = None
    
    def _expanding_zscore(self, series: pd.Series, min_periods: int = 36) -> pd.Series:
        """Z-score using expanding window (no future information)."""
        roll_mean = series.expanding(min_periods=min_periods).mean()
        roll_std = series.expanding(min_periods=min_periods).std()
        return (series - roll_mean) / roll_std.replace(0, np.nan)
    
    def fit_transform(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Compute the regime indicator from a DataFrame containing:
        - China_YoY: China CPI/export price YoY change
        - RealYield_Vol: rolling volatility of real yields
        - ULC_YoY: unit labor cost YoY change
        
        Returns the input DataFrame augmented with component scores
        and the composite RegimeScore.
        """
        result = df.copy()
        
        # Component 1: Globalization pressure
        # Higher China CPI growth = more inflationary = positive z-score
        result['z_china'] = self._expanding_zscore(df['China_YoY'])
        
        # Component 2: Real yield volatility
        # Higher vol = more financial stress = positive z-score
        result['z_ryv'] = self._expanding_zscore(df['RealYield_Vol'])
        
        # Component 3: Labor cost pressure
        # Higher ULC growth = tighter labor = positive z-score
        result['z_ulc'] = self._expanding_zscore(df['ULC_YoY'])
        
        # Composite: average of z-scores, passed through tanh for bounding
        z_avg = result[['z_china', 'z_ryv', 'z_ulc']].mean(axis=1)
        result['RegimeScore'] = np.tanh(z_avg)
        
        # Store for inspection
        self.components_ = result[['z_china', 'z_ryv', 'z_ulc']].copy()
        self.score_ = result['RegimeScore'].copy()
        
        return result
    
    def classify(self, score: float) -> str:
        """Human-readable regime classification."""
        if score < -0.3:
            return 'Deflationary Productivity Regime'
        elif score > 0.3:
            return 'Inflationary Productivity Regime'
        else:
            return 'Neutral / Transitional'


# Run the indicator
pri = ProductivityRegimeIndicator(lookback_months=60)
macro = pri.fit_transform(macro)
macro = macro.dropna(subset=['RegimeScore'])

print("ProductivityRegimeIndicator computed.")
print(f"\nCurrent regime score: {macro['RegimeScore'].iloc[-1]:.3f}")
print(f"Current classification: {pri.classify(macro['RegimeScore'].iloc[-1])}")
print(f"\n1990s average score: {macro.loc['1993':'2000', 'RegimeScore'].mean():.3f}")
print(f"2022-present average score: {macro.loc['2022':, 'RegimeScore'].mean():.3f}")

In [ ]:
# ============================================================
# VISUALIZATION 1: Regime Indicator Time Series
# ============================================================

fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True,
                          gridspec_kw={'height_ratios': [2, 1.5, 1.5], 'hspace': 0.08})

# --- Panel 1: Regime Score over time ---
ax1 = axes[0]
score = macro['RegimeScore']

# Color-fill based on regime
ax1.fill_between(score.index, score, 0, where=(score < 0),
                  color=C_BLUE, alpha=0.4, label='Deflationary regime')
ax1.fill_between(score.index, score, 0, where=(score >= 0),
                  color=C_RED, alpha=0.4, label='Inflationary regime')
ax1.plot(score.index, score, color=C_WHITE, linewidth=1.0, alpha=0.8)
ax1.axhline(0, color=C_GRAY, linewidth=0.5, linestyle='--')
ax1.axhline(0.3, color=C_RED, linewidth=0.5, linestyle=':', alpha=0.5)
ax1.axhline(-0.3, color=C_BLUE, linewidth=0.5, linestyle=':', alpha=0.5)

# Annotate key periods
ax1.annotate('1990s Globalization\nBoom', xy=(pd.Timestamp('1997-01-01'), -0.45),
             fontsize=9, color=C_BLUE, ha='center', weight='bold')
ax1.annotate('Post-COVID\nInflationary Regime', xy=(pd.Timestamp('2022-06-01'), 0.55),
             fontsize=9, color=C_RED, ha='center', weight='bold')

ax1.set_ylabel('Regime Score', fontsize=11)
ax1.set_title('Productivity Regime Indicator: Deflationary (-1) to Inflationary (+1)',
              fontsize=13, color=C_WHITE, pad=15)
ax1.legend(loc='upper left', fontsize=9, framealpha=0.3)
ax1.set_ylim(-1.05, 1.05)
ax1.grid(True, alpha=0.3)

# --- Panel 2: Component breakdown ---
ax2 = axes[1]
ax2.plot(macro.index, macro['z_china'], color=C_ORANGE, linewidth=1.0, label='China CPI pressure', alpha=0.8)
ax2.plot(macro.index, macro['z_ryv'], color=C_PURPLE, linewidth=1.0, label='Real yield volatility', alpha=0.8)
ax2.plot(macro.index, macro['z_ulc'], color=C_GREEN, linewidth=1.0, label='Unit labor costs', alpha=0.8)
ax2.axhline(0, color=C_GRAY, linewidth=0.5, linestyle='--')
ax2.set_ylabel('Z-Score', fontsize=11)
ax2.set_title('Component Z-Scores', fontsize=11, color=C_GRAY)
ax2.legend(loc='upper left', fontsize=8, framealpha=0.3)
ax2.grid(True, alpha=0.3)

# --- Panel 3: S&P 500 ---
ax3 = axes[2]
ax3.plot(macro.index, macro['SP500'], color=C_YELLOW, linewidth=1.2)
ax3.set_ylabel('S&P 500', fontsize=11)
ax3.set_yscale('log')
ax3.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))
ax3.grid(True, alpha=0.3)

# Shade regime periods
for ax in axes:
    ax.axvspan(pd.Timestamp('1993-01-01'), pd.Timestamp('2000-03-01'),
               alpha=0.08, color=C_BLUE)
    ax.axvspan(pd.Timestamp('2020-03-01'), macro.index[-1],
               alpha=0.08, color=C_RED)

fig.suptitle('Daily Observations: The Productivity Mirage', fontsize=15,
             color=C_ORANGE, y=0.98, weight='bold')
plt.tight_layout()
plt.savefig('regime_indicator.png', bbox_inches='tight', facecolor='#0a0a0a')
plt.show()
print("Figure saved: regime_indicator.png")

In [ ]:
# ============================================================
# VISUALIZATION 2: CAPE vs. Regime Indicator
# ============================================================

fig, ax1 = plt.subplots(figsize=(14, 7))

# CAPE on left axis
color_cape = C_YELLOW
ax1.plot(macro.index, macro['CAPE'], color=color_cape, linewidth=1.5, label='Shiller CAPE')
ax1.set_ylabel('Shiller CAPE Ratio', color=color_cape, fontsize=12)
ax1.tick_params(axis='y', labelcolor=color_cape)
ax1.set_ylim(10, 50)
ax1.grid(True, alpha=0.2)

# Regime score on right axis (inverted so deflationary = high = aligns with CAPE expansion)
ax2 = ax1.twinx()
ax2.plot(macro.index, -macro['RegimeScore'], color=C_BLUE, linewidth=1.2, alpha=0.7,
         label='Regime Indicator (inverted)')
ax2.fill_between(macro.index, -macro['RegimeScore'], 0,
                  where=(-macro['RegimeScore'] > 0), color=C_BLUE, alpha=0.15)
ax2.fill_between(macro.index, -macro['RegimeScore'], 0,
                  where=(-macro['RegimeScore'] <= 0), color=C_RED, alpha=0.15)
ax2.set_ylabel('Regime Indicator (inverted: up = deflationary)', color=C_BLUE, fontsize=12)
ax2.tick_params(axis='y', labelcolor=C_BLUE)
ax2.set_ylim(-1.2, 1.2)

# Annotations
ax1.annotate('1990s: Deflationary regime\n→ CAPE expansion to 44x',
             xy=(pd.Timestamp('1999-01-01'), 42), fontsize=9, color=C_BLUE,
             ha='center', weight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='#0a0a0a', edgecolor=C_BLUE, alpha=0.8))

ax1.annotate('2022: Inflationary regime\n→ CAPE compression to 28x',
             xy=(pd.Timestamp('2022-06-01'), 25), fontsize=9, color=C_RED,
             ha='center', weight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='#0a0a0a', edgecolor=C_RED, alpha=0.8))

ax1.annotate('Today: Regime says inflation\nbut CAPE is re-expanding →\nVULNERABILITY',
             xy=(pd.Timestamp('2025-01-01'), 37), fontsize=9, color=C_ORANGE,
             ha='center', weight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='#0a0a0a', edgecolor=C_ORANGE, alpha=0.8))

# Correlation
valid = macro[['CAPE', 'RegimeScore']].dropna()
corr, pval = pearsonr(-valid['RegimeScore'], valid['CAPE'])
ax1.text(0.02, 0.95, f'Correlation (CAPE vs inverted Regime): {corr:.2f} (p={pval:.4f})',
         transform=ax1.transAxes, fontsize=9, color=C_GRAY,
         verticalalignment='top')

# Legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=9, framealpha=0.3)

ax1.set_title('The Key Chart: Equity Multiples Track the Productivity Regime, Not Technology Alone',
              fontsize=13, color=C_WHITE, pad=15)
fig.suptitle('Daily Observations: The Productivity Mirage', fontsize=15,
             color=C_ORANGE, y=1.0, weight='bold')

plt.tight_layout()
plt.savefig('cape_vs_regime.png', bbox_inches='tight', facecolor='#0a0a0a')
plt.show()
print("Figure saved: cape_vs_regime.png")

In [ ]:
# ============================================================
# VISUALIZATION 3: Regime Comparison — 1990s vs. 2020s
# ============================================================

# Define the two regimes
regime_90s = macro.loc['1993':'2000']
regime_now = macro.loc['2020':]

fig, axes = plt.subplots(2, 3, figsize=(16, 9))

metrics = [
    ('PCE_YoY', 'PCE Inflation (YoY)', '{:.1%}'),
    ('China_YoY', 'China CPI (YoY)', '{:.1%}'),
    ('ULC_YoY', 'Unit Labor Costs (YoY)', '{:.1%}'),
    ('RealYield', 'Real Yield (10Y - PCE)', '{:.1%}'),
    ('RealYield_Vol', 'Real Yield Volatility', '{:.2f}'),
    ('RegimeScore', 'Regime Score', '{:.2f}'),
]

for idx, (col, title, fmt) in enumerate(metrics):
    ax = axes[idx // 3][idx % 3]
    
    vals_90 = regime_90s[col].dropna()
    vals_now = regime_now[col].dropna()
    
    bp = ax.boxplot([vals_90.values, vals_now.values],
                     labels=['1993-2000', '2020-Present'],
                     patch_artist=True,
                     boxprops=dict(facecolor='#1a1a1a', edgecolor=C_GRAY),
                     medianprops=dict(color=C_ORANGE, linewidth=2),
                     whiskerprops=dict(color=C_GRAY),
                     capprops=dict(color=C_GRAY),
                     flierprops=dict(markerfacecolor=C_RED, markersize=4))
    
    bp['boxes'][0].set_facecolor('#1a3a5c')
    bp['boxes'][1].set_facecolor('#5c1a1a')
    
    ax.set_title(title, fontsize=10, color=C_WHITE)
    
    # Add median labels
    for i, data in enumerate([vals_90, vals_now], 1):
        med = data.median()
        ax.text(i, med, f' {fmt.format(med)}', fontsize=8, color=C_ORANGE,
                va='center', ha='left')
    ax.grid(True, alpha=0.2)

fig.suptitle('Regime Comparison: 1993-2000 vs. 2020-Present',
             fontsize=14, color=C_ORANGE, weight='bold')
plt.tight_layout()
plt.savefig('regime_comparison.png', bbox_inches='tight', facecolor='#0a0a0a')
plt.show()
print("Figure saved: regime_comparison.png")

In [ ]:
# ============================================================
# STRESS TEST / BACKTEST:
# "What if we applied a regime-adjusted CAPE to today's market?"
# ============================================================

# Methodology:
# 1. Estimate the "regime-fair" CAPE using a simple linear model:
#    CAPE = alpha + beta * RegimeScore (inverted)
# 2. Compare actual CAPE to regime-fair CAPE
# 3. Derive an implied S&P 500 level if CAPE mean-reverted to regime-fair value

# Fit the relationship (full sample — this is an explanatory model, not a prediction)
from numpy.polynomial import polynomial as P

valid_data = macro[['CAPE', 'RegimeScore']].dropna()
x = -valid_data['RegimeScore'].values  # Inverted: positive = deflationary = higher multiples
y = valid_data['CAPE'].values

# Simple OLS
coeffs = np.polyfit(x, y, 1)
slope, intercept = coeffs[0], coeffs[1]

print(f"Regime-CAPE relationship: CAPE = {intercept:.1f} + {slope:.1f} * (-RegimeScore)")
print(f"  Interpretation: A 1-unit move toward deflationary regime → +{slope:.1f}x CAPE")

# Calculate regime-fair CAPE
macro['CAPE_Fair'] = intercept + slope * (-macro['RegimeScore'])

# Current values
current_cape = macro['CAPE'].iloc[-1]
current_fair = macro['CAPE_Fair'].iloc[-1]
current_sp500 = macro['SP500'].iloc[-1]
current_regime = macro['RegimeScore'].iloc[-1]

# Implied S&P 500 if CAPE adjusts to regime-fair value
# S&P_implied = S&P_actual * (CAPE_fair / CAPE_actual)
sp500_implied = current_sp500 * (current_fair / current_cape)
drawdown = (sp500_implied / current_sp500 - 1) * 100

print(f"\n{'='*60}")
print(f"STRESS TEST RESULTS")
print(f"{'='*60}")
print(f"Current S&P 500:          {current_sp500:>10,.0f}")
print(f"Current CAPE:             {current_cape:>10.1f}x")
print(f"Current Regime Score:     {current_regime:>10.3f} ({pri.classify(current_regime)})")
print(f"Regime-Fair CAPE:         {current_fair:>10.1f}x")
print(f"Regime-Adjusted S&P 500:  {sp500_implied:>10,.0f}")
print(f"Implied Repricing:        {drawdown:>10.1f}%")
print(f"{'='*60}")
print(f"\nConclusion: If equity multiples were priced consistently with the")
print(f"current productivity regime (as they were in past cycles), the")
print(f"S&P 500 would be approximately {abs(drawdown):.0f}% {'lower' if drawdown < 0 else 'higher'} than current levels.")

In [ ]:
# ============================================================
# VISUALIZATION 4: Stress Test — Actual vs. Regime-Fair Valuation
# ============================================================

fig = plt.figure(figsize=(14, 10))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# --- Panel A: Actual vs. Fair CAPE over time ---
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(macro.index, macro['CAPE'], color=C_YELLOW, linewidth=1.5, label='Actual CAPE')
ax1.plot(macro.index, macro['CAPE_Fair'], color=C_BLUE, linewidth=1.5,
         linestyle='--', label='Regime-Fair CAPE')

# Shade overvaluation
ax1.fill_between(macro.index, macro['CAPE'], macro['CAPE_Fair'],
                  where=(macro['CAPE'] > macro['CAPE_Fair']),
                  color=C_RED, alpha=0.2, label='Overvalued vs. regime')
ax1.fill_between(macro.index, macro['CAPE'], macro['CAPE_Fair'],
                  where=(macro['CAPE'] <= macro['CAPE_Fair']),
                  color=C_GREEN, alpha=0.2, label='Undervalued vs. regime')

ax1.set_ylabel('Shiller CAPE', fontsize=11)
ax1.set_title('Actual CAPE vs. Regime-Implied Fair CAPE', fontsize=13, color=C_WHITE)
ax1.legend(loc='upper left', fontsize=9, framealpha=0.3)
ax1.grid(True, alpha=0.2)
ax1.set_ylim(10, 50)

# --- Panel B: Scatter — Regime Score vs. CAPE ---
ax2 = fig.add_subplot(gs[1, 0])

# Color by era
mask_90s = (macro.index >= '1993-01-01') & (macro.index <= '2000-03-01')
mask_now = macro.index >= '2020-03-01'
mask_other = ~mask_90s & ~mask_now

ax2.scatter(-macro.loc[mask_other, 'RegimeScore'], macro.loc[mask_other, 'CAPE'],
            color=C_GRAY, alpha=0.3, s=15, label='Other periods')
ax2.scatter(-macro.loc[mask_90s, 'RegimeScore'], macro.loc[mask_90s, 'CAPE'],
            color=C_BLUE, alpha=0.7, s=25, label='1993-2000')
ax2.scatter(-macro.loc[mask_now, 'RegimeScore'], macro.loc[mask_now, 'CAPE'],
            color=C_RED, alpha=0.7, s=25, label='2020-Present')

# Fit line
x_fit = np.linspace(-1, 1, 100)
y_fit = intercept + slope * x_fit
ax2.plot(x_fit, y_fit, color=C_ORANGE, linewidth=2, linestyle='--', label='Best fit')

# Mark current
ax2.scatter(-current_regime, current_cape, color=C_ORANGE, s=120,
            marker='*', zorder=5, label='Current')

ax2.set_xlabel('Regime Score (inverted: right = deflationary)', fontsize=10)
ax2.set_ylabel('Shiller CAPE', fontsize=10)
ax2.set_title('Regime Score vs. CAPE', fontsize=11, color=C_WHITE)
ax2.legend(fontsize=8, framealpha=0.3)
ax2.grid(True, alpha=0.2)

# --- Panel C: Implied S&P 500 drawdown over time ---
ax3 = fig.add_subplot(gs[1, 1])
mispricing = ((macro['CAPE'] / macro['CAPE_Fair']) - 1) * 100
ax3.fill_between(macro.index, mispricing, 0,
                  where=(mispricing > 0), color=C_RED, alpha=0.4)
ax3.fill_between(macro.index, mispricing, 0,
                  where=(mispricing <= 0), color=C_GREEN, alpha=0.4)
ax3.plot(macro.index, mispricing, color=C_WHITE, linewidth=0.8)
ax3.axhline(0, color=C_GRAY, linewidth=0.5)

ax3.set_ylabel('CAPE Mispricing (%)', fontsize=10)
ax3.set_title('Overvaluation vs. Regime-Fair CAPE (%)', fontsize=11, color=C_WHITE)
ax3.grid(True, alpha=0.2)

# Annotate current
current_misprice = mispricing.iloc[-1]
ax3.annotate(f'Current: {current_misprice:+.0f}%',
             xy=(macro.index[-1], current_misprice),
             fontsize=9, color=C_ORANGE, weight='bold',
             xytext=(-80, 15), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', color=C_ORANGE))

fig.suptitle('Stress Test: Where Should the Market Be, Given the Regime?',
             fontsize=14, color=C_ORANGE, weight='bold', y=1.01)
plt.savefig('stress_test.png', bbox_inches='tight', facecolor='#0a0a0a')
plt.show()
print("Figure saved: stress_test.png")

In [ ]:
# ============================================================
# VISUALIZATION 5: Rolling Backtest — Signal Performance
# ============================================================

# Does the regime indicator have predictive power for forward equity returns?
# Test: sort months by regime score, compare forward 12-month S&P 500 returns.

macro['SP500_Fwd12m'] = macro['SP500'].pct_change(12).shift(-12)

# Split into quintiles by regime score
valid_bt = macro[['RegimeScore', 'SP500_Fwd12m', 'CAPE']].dropna()
valid_bt['Regime_Quintile'] = pd.qcut(valid_bt['RegimeScore'], 5,
                                        labels=['Q1\n(Most Deflationary)',
                                                'Q2', 'Q3', 'Q4',
                                                'Q5\n(Most Inflationary)'])

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# --- Panel A: Forward returns by regime quintile ---
ax1 = axes[0]
quintile_returns = valid_bt.groupby('Regime_Quintile', observed=True)['SP500_Fwd12m'].mean()
colors = [C_BLUE, '#3498DB', C_GRAY, '#E67E22', C_RED]
bars = ax1.bar(range(5), quintile_returns.values * 100, color=colors, edgecolor='#333', width=0.7)
ax1.set_xticks(range(5))
ax1.set_xticklabels(quintile_returns.index, fontsize=8)
ax1.set_ylabel('Avg. Forward 12-Month S&P 500 Return (%)', fontsize=10)
ax1.set_title('Forward Returns by Regime Quintile', fontsize=12, color=C_WHITE)
ax1.axhline(0, color=C_GRAY, linewidth=0.5)
ax1.grid(True, alpha=0.2, axis='y')

for bar, val in zip(bars, quintile_returns.values * 100):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             f'{val:.1f}%', ha='center', fontsize=9, color=C_WHITE)

# --- Panel B: Average CAPE by quintile ---
ax2 = axes[1]
quintile_cape = valid_bt.groupby('Regime_Quintile', observed=True)['CAPE'].mean()
bars2 = ax2.bar(range(5), quintile_cape.values, color=colors, edgecolor='#333', width=0.7)
ax2.set_xticks(range(5))
ax2.set_xticklabels(quintile_cape.index, fontsize=8)
ax2.set_ylabel('Average Shiller CAPE', fontsize=10)
ax2.set_title('Valuation by Regime Quintile', fontsize=12, color=C_WHITE)
ax2.grid(True, alpha=0.2, axis='y')

for bar, val in zip(bars2, quintile_cape.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             f'{val:.1f}x', ha='center', fontsize=9, color=C_WHITE)

fig.suptitle('Backtest: The Regime Indicator Predicts Both Returns and Multiples',
             fontsize=13, color=C_ORANGE, weight='bold')
plt.tight_layout()
plt.savefig('backtest_quintiles.png', bbox_inches='tight', facecolor='#0a0a0a')
plt.show()
print("Figure saved: backtest_quintiles.png")

In [ ]:
# ============================================================
# VISUALIZATION 6: AI Stocks vs. Regime — The Disconnect
# ============================================================

fig, ax1 = plt.subplots(figsize=(14, 6))

# SMH (semiconductor ETF) normalized to 100 at start of 2020
smh_plot = smh_monthly.loc['2018':].copy()
smh_norm = (smh_plot / smh_plot.iloc[0]) * 100

regime_plot = macro.loc['2018':, 'RegimeScore'].copy()

ax1.plot(smh_norm.index, smh_norm, color=C_GREEN, linewidth=2, label='SMH (Semiconductors, indexed)')
ax1.set_ylabel('SMH (indexed to 100)', color=C_GREEN, fontsize=11)
ax1.tick_params(axis='y', labelcolor=C_GREEN)
ax1.grid(True, alpha=0.2)

ax2 = ax1.twinx()
ax2.plot(regime_plot.index, regime_plot, color=C_RED, linewidth=1.5, alpha=0.8,
         label='Regime Score')
ax2.axhline(0, color=C_GRAY, linewidth=0.5, linestyle='--')
ax2.set_ylabel('Regime Score (>0 = Inflationary)', color=C_RED, fontsize=11)
ax2.tick_params(axis='y', labelcolor=C_RED)
ax2.set_ylim(-1, 1.5)

ax1.annotate('AI stocks surging\nwhile regime is inflationary\n→ DIVERGENCE',
             xy=(pd.Timestamp('2024-06-01'), smh_norm.loc['2024-06-01':].iloc[0]),
             fontsize=9, color=C_ORANGE, weight='bold', ha='center',
             xytext=(0, 40), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', color=C_ORANGE),
             bbox=dict(boxstyle='round,pad=0.3', facecolor='#0a0a0a', edgecolor=C_ORANGE, alpha=0.8))

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=9, framealpha=0.3)

ax1.set_title('The Disconnect: AI Stock Prices vs. The Macro Regime They Need',
              fontsize=13, color=C_WHITE, pad=15)
plt.tight_layout()
plt.savefig('ai_vs_regime.png', bbox_inches='tight', facecolor='#0a0a0a')
plt.show()
print("Figure saved: ai_vs_regime.png")

In [ ]:
# ============================================================
# SUMMARY STATISTICS TABLE
# ============================================================

def regime_summary(df, label):
    """Compute summary statistics for a regime period."""
    return pd.Series({
        'Period': label,
        'Regime Score (mean)': f"{df['RegimeScore'].mean():.3f}",
        'PCE Inflation (median)': f"{df['PCE_YoY'].median():.1%}",
        'China CPI YoY (median)': f"{df['China_YoY'].median():.1%}",
        'ULC YoY (median)': f"{df['ULC_YoY'].median():.1%}",
        'Real Yield (median)': f"{df['RealYield'].median():.1%}",
        'Real Yield Vol (median)': f"{df['RealYield_Vol'].median():.3f}",
        'CAPE (mean)': f"{df['CAPE'].mean():.1f}x",
        'S&P 500 Ann. Return': f"{((df['SP500'].iloc[-1]/df['SP500'].iloc[0])**(12/len(df))-1):.1%}",
    })

summary = pd.DataFrame([
    regime_summary(macro.loc['1993':'2000'], '1993-2000 (Deflationary Boom)'),
    regime_summary(macro.loc['2020':], '2020-Present (Inflationary AI)'),
]).set_index('Period')

print("\n" + "="*80)
print("REGIME COMPARISON SUMMARY")
print("="*80)
print(summary.to_string())
print("="*80)

---

## Section 4: Market Implications

### What This Analysis Shows

1. **The 1990s were not a technology story — they were a globalization story.** Our Regime Indicator was deeply negative (deflationary) throughout the 1990s, driven by falling Chinese export prices, low real yield volatility, and contained labor costs. This deflationary tailwind allowed the Fed to remain accommodative even as growth accelerated, which in turn justified equity multiple expansion.

2. **The current environment is structurally inflationary.** The Regime Indicator has been persistently positive since 2021. All three components — globalization pressure, real yield volatility, and labor costs — are working against the disinflationary transmission mechanism that made the 1990s special.

3. **CAPE is mispriced relative to the regime.** Our stress test shows that if equity multiples tracked the productivity regime as they have historically, the S&P 500 would be materially lower. The current pricing implies investors are betting on a 1990s-style regime that the data does not support.

4. **AI stocks are especially vulnerable.** Semiconductor stocks have re-rated to levels that assume AI will deliver 1990s-style productivity gains to the *entire economy*. But the macro regime means those productivity gains — even if they materialize — will be offset by higher input costs, tighter labor, and less accommodative monetary policy.

### What Could Prove Us Wrong

Intellectual honesty requires stating the conditions under which this thesis fails:

- **AI productivity gains are so large they overwhelm inflation.** If AI reduces service-sector costs by 30%+ within 3-5 years (not just tech sector), the inflationary regime could flip. We think this is unlikely on that timeframe.
- **A major geopolitical de-escalation.** If US-China tensions resolve and globalization resumes, the China CPI component would reverse. Possible but not the base case.
- **A deep recession.** A severe downturn would crush demand and labor costs, temporarily restoring a deflationary regime. But this would also destroy earnings, so multiples wouldn't necessarily expand.
- **Fiscal consolidation.** If the US undertakes meaningful deficit reduction, real yield volatility would fall. There is no political constituency for this.

### Positioning Implications

For a systematic macro portfolio:
- **Underweight long-duration equities** (high P/E growth stocks, including AI beneficiaries) relative to consensus
- **Favor real assets and pricing-power equities** that benefit from nominal growth
- **Stay structurally short duration** in fixed income — the regime does not support a return to 1990s-style real yield compression
- **Monitor the Regime Indicator for regime change signals** — a sustained move below 0 would challenge this thesis

---

*The core message is simple: the market is pricing AI as if it will recreate the 1990s. But the 1990s were not about technology — they were about a deflationary world order that no longer exists. What the market is really betting on is not AI, but a return to globalization. And that bet, in our view, is wrong.*

---

**Methodology Note:** All indicators use trailing data only (no look-ahead bias). The Regime Indicator uses expanding-window z-scores, meaning each observation is scored relative to all prior data available at that point. The CAPE-regime regression uses the full sample for illustration; in a production system, this would use rolling out-of-sample estimation.

**Disclaimer:** This is a research exercise and does not constitute investment advice. The analysis uses proxy data where official FRED data is unavailable; results should be validated with authoritative data sources before any investment application.

In [ ]:
print("\n" + "="*80)
print("PROJECT COMPLETE")
print("="*80)
print("\nFiles generated:")
print("  - regime_indicator.png     : Regime score time series with components")
print("  - cape_vs_regime.png       : CAPE overlaid with regime indicator")
print("  - regime_comparison.png    : Box plots comparing 1990s vs 2020s")
print("  - stress_test.png          : Actual vs regime-fair valuation")
print("  - backtest_quintiles.png   : Forward returns and CAPE by regime quintile")
print("  - ai_vs_regime.png         : Semiconductor stocks vs regime disconnect")
print("\nProductivityRegimeIndicator class is available for reuse.")
print("Set FRED_API_KEY environment variable for authoritative FRED data.")